In [1]:
import os
os.environ['HF_HOME'] = '/rds/general/project/genomics-facility-archive-2019/live/adatta17/test_dir/test142_gpt/cache/'

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

/home/adatta17/anaconda3/envs/haystack/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_current_temperature(location: str, unit: str) -> float:
    """
    Get the current temperature at a location.
    
    Args:
        location: The location to get the temperature for, in the format "City, Country"
        unit: The unit to return the temperature in. (choices: ["celsius", "fahrenheit"])
    Returns:
        The current temperature at the specified location in the specified units, as a float.
    """
    return 22.  # A real function should probably actually get the temperature!

def get_current_wind_speed(location: str) -> float:
    """
    Get the current wind speed in km/h at a given location.
    
    Args:
        location: The location to get the temperature for, in the format "City, Country"
    Returns:
        The current wind speed at the given location in km/h, as a float.
    """
    return 6.  # A real function should probably actually get the wind speed!

tools = [get_current_temperature, get_current_wind_speed]

In [4]:
# messages = [
#   {"role": "system", "content": "You are a bot that responds to weather queries. You should reply with the unit used in the queried location."},
#   {"role": "user", "content": "Hey, what's the temperature in Paris in celsius right now?"}
# ]

In [5]:
# inputs = tokenizer.apply_chat_template(messages, tools=tools, add_generation_prompt=True, return_dict=True, return_tensors="pt")
# inputs = {k: v.to(model.device) for k, v in inputs.items()}
# out = model.generate(**inputs, max_new_tokens=128)
# print(tokenizer.decode(out[0][len(inputs["input_ids"][0]):]))

In [20]:
import json

class Conversation:
    def __init__(self, tokenizer: AutoTokenizer, model: AutoModelForCausalLM, tools: list):
        self.tokenizer = tokenizer #AutoTokenizer.from_pretrained(model_name)
        self.model = model #AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
        self.messages = [
            {"role": "system",
             "content": "You are a bot that responds to weather queries. You should reply with the unit used in the queried location."}
        ]
        self.tools = tools
    
    def _tokenize_input(self) -> dict:
        inputs = \
            self.tokenizer.apply_chat_template(
                self.messages,
                tools=self.tools,
                add_generation_prompt=True,
                return_dict=True,
                return_tensors="pt")
        inputs = {k: v.to(model.device) for k, v in inputs.items()}
        return inputs

    def _get_model_output(self, max_new_tokens: int = 512) -> str:
        inputs = self._tokenize_input()
        output = \
            self.model.generate(**inputs, max_new_tokens=max_new_tokens)
        response = \
            tokenizer.decode(output[0][len(inputs["input_ids"][0]):])
        return response

    def _call_function(self, response):
        response = \
            response.replace("<|im_end|>", "")
        response = \
            response.\
                replace("<tool_call>", "").\
                replace("</tool_call>", "").\
                strip()
        tools_json = json.loads(response)
        output = globals()[tools_json["name"]](**tools_json["arguments"])
        tool_call = json.dumps(tools_json)
        self.messages.append({"role": "assistant", "tool_calls": [{"type": "function", "function": tool_call}]})
        self.messages.append({"role": "tool", "name": tools_json["name"], "content": f"{output}"})
        response = \
            self._get_model_output()
        response = \
            response.replace("<|im_end|>", "")
        return response
        
    def chat(self, message: str):
        self.messages.append({
            "role": "user", "content": message})
        response = \
            self._get_model_output()
        if response.startswith("<tool_call>"):
            tool_response = \
                self._call_function(response)
            self.messages.append({"role": "assistant", "content": tool_response})
            return tool_response
        else:
            self.messages.append({"role": "assistant", "content": response})
            return response
            

In [14]:
checkpoint = "NousResearch/Hermes-2-Pro-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards: 100%|██████████| 4/4 [01:39<00:00, 24.99s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [21]:
conv = Conversation(tokenizer=tokenizer, model=model, tools=[get_current_temperature, get_current_wind_speed])

In [22]:
conv.chat("Hey, what's the temperature in Paris in celsius right now?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'The current temperature in Paris, France is 22.0°C (71.6°F).'

In [23]:
conv.chat("Hey, what's the wind speed?")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'The current wind speed in Paris, France is 6.0 km/h. Please note that the wind speed is not commonly measured in Fahrenheit, so the unit used is kilometers per hour (km/h).'

https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_2#-spinquant-